In [ ]:
import os
import librosa
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# 1. DEFINISI PATH DATASET
DATASET_PATH = "dataset/cats_dogs" 
CATEGORIES = ["cats", "dogs"]

# 2. FUNGSI EKSTRAKSI FITUR (MFCC)
def extract_features(file_path):
    try:
        # Load audio file 
        audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast', duration=10) 
        
        # Ekstraksi MFCC features
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        
        # Ambil rata-rata MFCC 
        mfccs_processed = np.mean(mfccs.T, axis=0)
        return mfccs_processed
    except Exception as e:
        print(f"Error parsing {file_path}: {e}")
        return None

# 3. LOAD DATA & PREPROCESSING
features = []
labels = []

print("Mulai memproses dataset... (Bisa memakan waktu)")

for category in CATEGORIES:
    path = os.path.join(DATASET_PATH, category)
    if not os.path.exists(path):
        path = os.path.join("dataset", f"{category}_split", category) 
        print(f"Mencari di path alternatif: {path}")

    class_num = CATEGORIES.index(category) 
    
    for img in os.listdir(path):
        file_path = os.path.join(path, img)
        data = extract_features(file_path)
        if data is not None:
            features.append(data)
            labels.append(class_num)

print(f"Selesai memproses {len(features)} file audio.")

# Konversi ke Numpy Array
X = np.array(features)
y = np.array(labels)

# Split Data (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. MEMBANGUN MODEL (SIMPLE ANN)
model = Sequential()
model.add(Dense(256, input_shape=(40,), activation='relu')) # Input shape 40 karena n_mfcc=40
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # Output 1 neuron (Binary: 0 atau 1)

model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')

# 5. TRAINING MODEL
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# 6. EVALUASI
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Akurasi pada data test: {accuracy*100:.2f}%")

# 7. SIMPAN MODEL
model.save('audio_classifier_model.h5')
print("Model berhasil disimpan sebagai 'audio_classifier_model.h5'")